# Complete Setup Steps for LenaBean App

## 1. Initial Project and Dependencies
```bash
# Create project
npx create-expo-app LenaBeanMemories
cd LenaBeanMemories

# Create structure
mkdir -p app/(tabs)
mkdir -p app/utils
mkdir -p assets

# Install core dependencies
npm install @react-navigation/native @react-navigation/bottom-tabs
npx expo install react-native-screens react-native-safe-area-context
npx expo install @react-native-async-storage/async-storage
npx expo install expo-image-picker
npx expo install expo-notifications
npx expo install @expo/vector-icons react-native-gesture-handler react-native-reanimated
```

## 2. Create Core Files
Create and populate key files:

```bash
touch app/(tabs)/index.js
touch app/(tabs)/add.tsx
touch app/(tabs)/reminders.tsx
touch app/(tabs)/_layout.tsx
touch app/utils/storage.ts
touch app/types.ts
touch app/(tabs)/wisdombook.tsx
```

## 3. Setup Assets
```bash
# Create assets directory
mkdir -p assets

# Create placeholder images
curl https://picsum.photos/1024 -o assets/icon.png
curl https://picsum.photos/1024 -o assets/splash.png
```

## 4. Update Dependencies
```bash
# Install and update all required packages
npx expo install expo@~52.0.26 expo-blur@~14.0.3 expo-constants@~17.0.4 \
expo-font@~13.0.3 expo-haptics@~14.0.1 expo-linking@~7.0.4 \
expo-router@~4.0.17 expo-splash-screen@~0.29.21 expo-status-bar@~2.0.1 \
expo-symbols@~0.2.1 expo-system-ui@~4.0.7 expo-web-browser@~14.0.2 \
react-native@0.76.6 expo-image-picker@~16.0.4 expo-notifications@~0.29.12 \
jest-expo@~52.0.3 expo-dev-client
```

## 5. Development Build Setup
```bash
# Install EAS CLI
npm install -g eas-cli

# Configure and create build
eas login
eas build:configure
eas build --profile development --platform ios
```

## 6. Run Development Server
```bash
npx expo start --dev-client
```

# Here are the steps we added after the initial setup:

1. First build: `eas build --platform ios --profile preview`
2. Test UI changes: `npx expo start` (using Expo Go)
3. Build after notification changes: `eas build --platform ios --profile preview`

## 7. Notification Feature Setup
```javascript
// In reminders.tsx
- Added notification handler
- Set up daily notifications for 7:30 PM
- Added notification permission checks
- Implemented notification toggle with AsyncStorage
```

## 8. Added Wisdom Book Feature
```bash
# Install additional dependencies
npx expo install @expo/vector-icons react-native-gesture-handler react-native-reanimated
```
- Created new file `app/(tabs)/wisdombook.tsx`
- Modify `app/(tabs)/_layout.tsx` to add the new tab
- Implemented swipeable interface
- Added favorite functionality
- Added page counting and navigation

## 9. UI Improvements
```javascript
// Modified tab navigation in app/(tabs)/_layout.tsx
- Removed triangle indicators
- Reordered tabs (Wisdom, Memories, Add, Reminders)
- Added custom icons for each tab

// In reminders.tsx
- Changed from showing all tips to one random daily tip
```

## 10. Build Process
```bash
# For testing UI changes
npx expo start

# For building standalone app with notifications
eas build --platform ios --profile preview
```

## File Organization Overview

```
LenaBeanMemories/
├── app/
│   ├── (tabs)/
│   │   ├── index.js       # Main memories list
│   │   ├── add.tsx        # Add new memory form
│   │   ├── reminders.tsx  # Reminders and tips
│   │   ├── wisdombook.tsx # Wisdom book feature
│   │   └── _layout.tsx    # Tab navigation setup
│   ├── utils/
│   │   └── storage.ts     # AsyncStorage management
│   └── types.ts           # TypeScript definitions
├── assets/
│   ├── icon.png          # App icon
│   └── splash.png        # Splash screen
├── app.json              # App configuration
└── package.json          # Dependencies
```

# LenaBean App - Complete Code Documentation

## NOTE: The following codes are not the most updated versions; please see the files included in the repo outside of this notebook for the newest updates

## 1. Project Setup & Configuration

### app.json

```json
{
  "expo": {
    "name": "LenaBean",
    "slug": "LenaBeanMemories",
    "version": "1.0.0",
    "orientation": "portrait",
    "icon": "./assets/icon.png",
    "userInterfaceStyle": "light",
    "splash": {
      "image": "./assets/splash.png",
      "resizeMode": "contain",
      "backgroundColor": "#ffffff"
    },
    "scheme": "lenabean",
    "newArchEnabled": true,
    "ios": {
      "bundleIdentifier": "com.yiyi19911211.LenaBeanMemories",
      "buildNumber": "1",
      "supportsTablet": true
    },
    "plugins": [
      [
        "expo-image-picker",
        {
          "photosPermission": "Allow LenaBean to access your photos"
        }
      ]
    ]
  }
}
```

## 2. Core Files

### A. app/utils/types.ts

```typescript
export interface Memory {
  id: string;
  content: string;
  category: string;
  date: string;
  imageUri?: string;
}

export default Memory;
```

### B. app/utils/storage.ts

```typescript
import AsyncStorage from '@react-native-async-storage/async-storage';
import { Memory } from './types';

const STORAGE_KEY = 'lenabean_memories';

const storage = {
  async saveMemory(memory: Memory): Promise<void> {
    try {
      const memories = await AsyncStorage.getItem(STORAGE_KEY);
      console.log('Current memories:', memories);
      const existingMemories = memories ? JSON.parse(memories) : [];
      const updatedMemories = [memory, ...existingMemories];
      await AsyncStorage.setItem(STORAGE_KEY, JSON.stringify(updatedMemories));
      console.log('Saved successfully');
    } catch (error) {
      console.error('Storage error:', error);
      throw error;
    }
  },

  async getMemories(): Promise<Memory[]> {
    try {
      const memories = await AsyncStorage.getItem(STORAGE_KEY);
      return memories ? JSON.parse(memories) : [];
    } catch (error) {
      console.error('Error getting memories:', error);
      return [];
    }
  },

  async deleteMemory(id: string): Promise<void> {
    try {
      const memories = await this.getMemories();
      const updatedMemories = memories.filter(memory => memory.id !== id);
      await AsyncStorage.setItem(STORAGE_KEY, JSON.stringify(updatedMemories));
    } catch (error) {
      console.error('Error deleting memory:', error);
      throw error;
    }
  }
};

export default storage;
```

### C. app/(tabs)/_layout.tsx

```typescript
import { Tabs } from 'expo-router';

export default function TabLayout() {
  return (
    <Tabs
      screenOptions={{
        headerStyle: {
          backgroundColor: '#FFF5F7',
        },
        headerTitleStyle: {
          color: '#4A5568',
          fontSize: 20,
        },
        tabBarActiveTintColor: '#9F7AEA',
        tabBarInactiveTintColor: '#718096',
      }}
    >
      <Tabs.Screen
        name="index"
        options={{
          title: "Lena Bean",
          tabBarLabel: "Memories"
        }}
      />
      <Tabs.Screen
        name="add"
        options={{
          title: "New Memory",
          tabBarLabel: "Add"
        }}
      />
      <Tabs.Screen
        name="reminders"
        options={{
          title: "Reminders",
          tabBarLabel: "Reminders"
        }}
      />
    </Tabs>
  );
}
```

### D. app/(tabs)/index.js (Memories Screen)

```typescript
import React, { useState } from 'react';
import { 
  Text, 
  View, 
  StyleSheet, 
  ScrollView, 
  Image,
  TouchableOpacity,
  Alert
} from 'react-native';
import { useSafeAreaInsets } from 'react-native-safe-area-context';
import storage from '../utils/storage';
import { Memory } from '../utils/types';
import { useFocusEffect } from 'expo-router';

const getCategoryEmoji = (category: string) => {
  const emojis = {
    sweet: '💝',
    proud: '🌟',
    funny: '😄',
    milestone: '🎯',
    creative: '🎨',
    kind: '🫂',
  };
  return emojis[category] || '✨';
};

const formatDate = (dateString: string) => {
  const date = new Date(dateString);
  const options: Intl.DateTimeFormatOptions = { 
    year: 'numeric', 
    month: 'long', 
    day: 'numeric',
    hour: '2-digit',
    minute: '2-digit'
  };
  return date.toLocaleDateString('en-US', options);
};

const MemoryCard = ({ memory, onDelete }: { memory: Memory; onDelete: (id: string) => void }) => {
  const handleLongPress = () => {
    Alert.alert(
      'Delete Memory',
      'Are you sure you want to delete this memory?',
      [
        {
          text: 'Cancel',
          style: 'cancel'
        },
        {
          text: 'Delete',
          style: 'destructive',
          onPress: () => onDelete(memory.id)
        }
      ]
    );
  };

  return (
    <TouchableOpacity 
      style={styles.card}
      onLongPress={handleLongPress}
      delayLongPress={500}
    >
      <Text style={styles.date}>{formatDate(memory.date)}</Text>
      <Text style={styles.content}>{memory.content}</Text>
      {memory.imageUri && (
        <Image 
          source={{ uri: memory.imageUri }} 
          style={styles.memoryImage}
          resizeMode="cover"
        />
      )}
      <View style={styles.categoryTag}>
        <Text style={styles.categoryText}>
          {getCategoryEmoji(memory.category)} {memory.category}
        </Text>
      </View>
    </TouchableOpacity>
  );
};

export default function MemoriesScreen() {
  const insets = useSafeAreaInsets();
  const [memories, setMemories] = useState<Memory[]>([]);

  const loadMemories = async () => {
    const loadedMemories = await storage.getMemories();
    setMemories(loadedMemories);
  };

  useFocusEffect(
    React.useCallback(() => {
      loadMemories();
    }, [])
  );

  const handleDeleteMemory = async (id: string) => {
    try {
      await storage.deleteMemory(id);
      await loadMemories();
      Alert.alert('Success', 'Memory deleted successfully');
    } catch (error) {
      Alert.alert('Error', 'Failed to delete memory');
    }
  };

  return (
    <View style={[styles.container, { paddingTop: insets.top }]}>
      <ScrollView 
        style={styles.scrollView}
        contentContainerStyle={styles.scrollContent}
      >
        <Text style={styles.headerText}>Sweet Memories ✨</Text>
        {memories.length === 0 ? (
          <View style={styles.emptyContainer}>
            <Text style={styles.emptyText}>
              No memories yet. Add your first sweet moment! 💝
            </Text>
            <Text style={styles.emptySubText}>
              Tap the "Add" tab below to create a new memory.
            </Text>
          </View>
        ) : (
          memories.map(memory => (
            <MemoryCard 
              key={memory.id} 
              memory={memory}
              onDelete={handleDeleteMemory}
            />
          ))
        )}
      </ScrollView>
    </View>
  );
}

const styles = StyleSheet.create({
  container: {
    flex: 1,
    backgroundColor: '#FFF5F7',
  },
  scrollView: {
    flex: 1,
  },
  scrollContent: {
    padding: 16,
    paddingBottom: 32,
  },
  headerText: {
    fontSize: 24,
    fontWeight: 'bold',
    color: '#4A5568',
    marginBottom: 16,
    textAlign: 'center',
  },
  card: {
    backgroundColor: '#FFFFFF',
    borderRadius: 16,
    padding: 16,
    marginBottom: 16,
    shadowColor: '#718096',
    shadowOffset: { width: 0, height: 2 },
    shadowOpacity: 0.1,
    shadowRadius: 4,
    elevation: 2,
  },
  date: {
    fontSize: 14,
    color: '#718096',
    marginBottom: 8,
  },
  content: {
    fontSize: 16,
    color: '#4A5568',
    lineHeight: 24,
    marginBottom: 12,
  },
  memoryImage: {
    width: '100%',
    height: 200,
    borderRadius: 8,
    marginBottom: 12,
  },
  categoryTag: {
    backgroundColor: '#F7FAFC',
    paddingHorizontal: 12,
    paddingVertical: 6,
    borderRadius: 12,
    alignSelf: 'flex-start',
  },
  categoryText: {
    fontSize: 14,
    color: '#718096',
  },
  emptyContainer: {
    alignItems: 'center',
    marginTop: 64,
    paddingHorizontal: 24,
  },
  emptyText: {
    textAlign: 'center',
    color: '#718096',
    fontSize: 18,
    marginBottom: 8,
  },
  emptySubText: {
    textAlign: 'center',
    color: '#A0AEC0',
    fontSize: 14,
  },
});
```

### E. app/(tabs)/add.tsx

```typescript
import React, { useState, useEffect } from 'react';
import { 
  Text, 
  View, 
  StyleSheet, 
  TextInput, 
  TouchableOpacity, 
  ScrollView,
  Image,
  Alert
} from 'react-native';
import { useSafeAreaInsets } from 'react-native-safe-area-context';
import * as ImagePicker from 'expo-image-picker';
import { router } from 'expo-router';
import storage from '../utils/storage';

const categories = [
  { id: 'sweet', label: 'Sweet', emoji: '💝' },
  { id: 'funny', label: 'Funny', emoji: '😄' },
  { id: 'proud', label: 'Proud', emoji: '🌟' },
  { id: 'milestone', label: 'Milestone', emoji: '🎯' },
  { id: 'creative', label: 'Creative', emoji: '🎨' },
  { id: 'kind', label: 'Kind', emoji: '🫂' },
];

export default function AddMemoryScreen() {
  const insets = useSafeAreaInsets();
  const [memory, setMemory] = useState('');
  const [selectedCategory, setSelectedCategory] = useState(null);
  const [image, setImage] = useState(null);

  useEffect(() => {
    (async () => {
      const { status } = await ImagePicker.requestMediaLibraryPermissionsAsync();
      if (status !== 'granted') {
        Alert.alert('Permission needed', 'Please allow access to your photo library');
      }
    })();
  }, []);

  const pickImage = async () => {
    try {
      let result = await ImagePicker.launchImageLibraryAsync({
        mediaTypes: ImagePicker.MediaType.Images,
        allowsEditing: true,
        aspect: [4, 3],
        quality: 1,
      });

      if (!result.canceled) {
        setImage(result.assets[0].uri);
      }
    } catch (error) {
      console.error('Error picking image:', error);
      Alert.alert('Error', 'Failed to pick image. Please try again.');
    }
  };

  const handleSave = async () => {
    try {
      if (!memory || !selectedCategory) {
        Alert.alert('Missing Info', 'Please fill in the memory and select a category');
        return;
      }

      const newMemory = {
        id: Date.now().toString(),
        content: memory,
        category: selectedCategory,
        date: new Date().toISOString(),
        imageUri: image
      };

      await storage.saveMemory(newMemory);
      
      Alert.alert('Memory Saved!', 'Your sweet memory has been saved 💝', [
        {
          text: 'OK',
          onPress: () => {
            setMemory('');
            setSelectedCategory(null);
            setImage(null);
            router.push('/(tabs)/');
          }
        }
      ]);
    } catch (error) {
      console.error('Storage error:', error);
      Alert.alert('Error', 'Failed to save memory. Please try again.');
    }
  };

  return (
    <View style={[styles.container, { paddingTop: insets.top }]}>
      <ScrollView style={styles.scrollView}>
        <Text style={styles.title}>Capture a Sweet Moment ✨</Text>
        
        <View style={styles.inputContainer}>
          <TextInput
            style={styles.input}
            multiline
            placeholder="What did Lena Bean do today?"
            value={memory}
            onChangeText={setMemory}
            placeholderTextColor="#A0AEC0"
          />
        </View>

        <Text style={styles.sectionTitle}>Choose a Category</Text>
        <View style={styles.categoryContainer}>
          {categories.map(category => (
            <TouchableOpacity 
              key={category.id}
              style={[
                styles.categoryButton,
                selectedCategory === category.id && styles.selectedCategory
              ]}
              onPress={() => setSelectedCategory(category.id)}
            >
              <Text style={styles.categoryEmoji}>{category.emoji}</Text>
              <Text style={[
                styles.categoryLabel,
                selectedCategory === category.id && styles.selectedCategoryText
              ]}>
                {category.label}
              </Text>
            </TouchableOpacity>
          ))}
        </View>

        <TouchableOpacity style={styles.photoButton} onPress={pickImage}>
          <Text style={styles.photoButtonText}>
            {image ? '📸 Change Photo' : '📸 Add Photo'}
          </Text>
        </TouchableOpacity>

        {image && (
          <Image source={{ uri: image }} style={styles.previewImage} />
        )}

        <TouchableOpacity 
          style={[
            styles.saveButton,
            (!memory || !selectedCategory) && styles.saveButtonDisabled
          ]}
          onPress={handleSave}
          disabled={!memory || !selectedCategory}
        >
          <Text style={styles.saveButtonText}>Save Memory</Text>
        </TouchableOpacity>
      </ScrollView>
    </View>
  );
}

const styles = StyleSheet.create({
  container: {
    flex: 1,
    backgroundColor: '#FFF5F7',
  },
  scrollView: {
    padding: 16,
  },
  title: {
    fontSize: 24,
    fontWeight: 'bold',
    color: '#4A5568',
    marginBottom: 20,
    textAlign: 'center',
  },
  inputContainer: {
    backgroundColor: '#FFFFFF',
    borderRadius: 12,
    padding: 12,
    marginBottom: 20,
    shadowColor: '#718096',
    shadowOffset: { width: 0, height: 2 },
    shadowOpacity: 0.1,
    shadowRadius: 4,
  },
  input: {
    minHeight: 100,
    fontSize: 16,
    color: '#4A5568',
    textAlignVertical: 'top',
  },
  sectionTitle: {
    fontSize: 18,
    fontWeight: '600',
    color: '#4A5568',
    marginBottom: 12,
  },
  categoryContainer: {
    flexDirection: 'row',
    flexWrap: 'wrap',
    gap: 8,
    marginBottom: 20,
  },
  categoryButton: {
    backgroundColor: '#FFFFFF',
    borderRadius: 12,
    padding: 12,
    alignItems: 'center',
    width: '30%',
    shadowColor: '#718096',
    shadowOffset: { width: 0, height: 1 },
    shadowOpacity: 0.1,
    shadowRadius: 2,
  },
  selectedCategory: {
    backgroundColor: '#9F7AEA',
  },
  categoryEmoji: {
    fontSize: 24,
    marginBottom: 4,
  },
  categoryLabel: {
    fontSize: 12,
    color: '#4A5568',
  },
  selectedCategoryText: {
    color: '#FFFFFF',
  },
  photoButton: {
    backgroundColor: '#FFFFFF',
    borderRadius: 12,
    padding: 16,
    alignItems: 'center',
    marginBottom: 16,
  },
  photoButtonText: {
    fontSize: 16,
    color: '#4A5568',
  },
  previewImage: {
    width: '100%',
    height: 200,
    borderRadius: 12,
    marginBottom: 16,
  },
  saveButton: {
    backgroundColor: '#9F7AEA',
    borderRadius: 12,
    padding: 16,
    alignItems: 'center',
    marginBottom: 32,
  },
  saveButtonDisabled: {
    backgroundColor: '#CBD5E0',
  },
  saveButtonText: {
    fontSize: 16,
    color: '#FFFFFF',
    fontWeight: '600',
  },
});
```

Would you like me to continue with reminders.tsx?

### F. app/(tabs)/reminders.tsx

```typescript
import React, { useState, useEffect } from 'react';
import { 
  Text, 
  View, 
  StyleSheet, 
  ScrollView, 
  TouchableOpacity,
  Switch,
  Alert 
} from 'react-native';
import { useSafeAreaInsets } from 'react-native-safe-area-context';
import { getMemories, type Memory } from '../utils/storage';
import * as Notifications from 'expo-notifications';

const parentingTips = [
  {
    id: '1',
    title: "Learning Independence",
    message: "When she's taking forever to put on her shoes, remember: she's learning independence with each tiny struggle. Deep breath - she's not giving you a hard time, she's having a hard time.",
    emoji: '👟'
  },
  {
    id: '2',
    title: "Big Feelings",
    message: "During tantrums, remember that big emotions are hard for little hearts. You're her safe space to feel all feelings.",
    emoji: '💗'
  },
  {
    id: '3',
    title: "Growing Mind",
    message: "When she asks 'why' for the hundredth time, remember: her curiosity is building her understanding of the world.",
    emoji: '🌱'
  },
  {
    id: '4',
    title: "Little Helper",
    message: "When simple tasks take longer because she wants to help, remember: you're raising a person who wants to contribute.",
    emoji: '🌟'
  },
  {
    id: '5',
    title: "Gentle Reminder",
    message: "Your patience in her slow moments is teaching her it's okay to take the time she needs.",
    emoji: '🫂'
  }
];

const MemoryFlashback = ({ memory }) => (
  <View style={styles.flashbackCard}>
    <Text style={styles.flashbackDate}>On this day...</Text>
    <Text style={styles.flashbackContent}>{memory.content}</Text>
    <View style={styles.categoryTag}>
      <Text style={styles.categoryText}>
        {memory.category} ✨
      </Text>
    </View>
  </View>
);

const ParentingTipCard = ({ tip }) => (
  <View style={styles.tipCard}>
    <Text style={styles.tipEmoji}>{tip.emoji}</Text>
    <Text style={styles.tipTitle}>{tip.title}</Text>
    <Text style={styles.tipMessage}>{tip.message}</Text>
  </View>
);

export default function RemindersScreen() {
  const insets = useSafeAreaInsets();
  const [dailyTipsEnabled, setDailyTipsEnabled] = useState(false);
  const [flashbacksEnabled, setFlashbacksEnabled] = useState(false);
  const [todaysMemories, setTodaysMemories] = useState([]);

  useEffect(() => {
    checkAndLoadMemories();
    setupNotifications();
  }, []);

  const checkAndLoadMemories = async () => {
    const allMemories = await getMemories();
    const today = new Date();
    
    // Find memories from same day in previous years
    const memories = allMemories.filter(memory => {
      const memoryDate = new Date(memory.date);
      return memoryDate.getDate() === today.getDate() && 
             memoryDate.getMonth() === today.getMonth() &&
             memoryDate.getFullYear() < today.getFullYear();
    });
    
    setTodaysMemories(memories);
  };

  const setupNotifications = async () => {
    const { status } = await Notifications.requestPermissionsAsync();
    if (status !== 'granted') {
      Alert.alert('Permissions needed', 'Please enable notifications to receive daily reminders');
      return;
    }
  };

  const toggleDailyTips = async (enabled) => {
    setDailyTipsEnabled(enabled);
    if (enabled) {
      // Schedule daily parenting tip notification
      await Notifications.scheduleNotificationAsync({
        content: {
          title: "Daily Parenting Wisdom 💝",
          body: parentingTips[Math.floor(Math.random() * parentingTips.length)].message,
        },
        trigger: {
          hour: 9,
          minute: 0,
          repeats: true,
        },
      });
    } else {
      await Notifications.cancelAllScheduledNotificationsAsync();
    }
  };

  return (
    <View style={[styles.container, { paddingTop: insets.top }]}>
      <ScrollView style={styles.scrollView}>
        <Text style={styles.headerText}>Reminders & Reflections 💭</Text>

        <View style={styles.section}>
          <View style={styles.sectionHeader}>
            <Text style={styles.sectionTitle}>Memory Flashbacks</Text>
            <Switch
              value={flashbacksEnabled}
              onValueChange={setFlashbacksEnabled}
              trackColor={{ false: '#CBD5E0', true: '#9F7AEA' }}
            />
          </View>
          {todaysMemories.length > 0 ? (
            todaysMemories.map(memory => (
              <MemoryFlashback key={memory.id} memory={memory} />
            ))
          ) : (
            <Text style={styles.emptyText}>No memories from this day in previous years</Text>
          )}
        </View>

        <View style={styles.section}>
          <View style={styles.sectionHeader}>
            <Text style={styles.sectionTitle}>Daily Parenting Wisdom</Text>
            <Switch
              value={dailyTipsEnabled}
              onValueChange={toggleDailyTips}
              trackColor={{ false: '#CBD5E0', true: '#9F7AEA' }}
            />
          </View>
          {parentingTips.map(tip => (
            <ParentingTipCard key={tip.id} tip={tip} />
          ))}
        </View>
      </ScrollView>
    </View>
  );
}

const styles = StyleSheet.create({
  container: {
    flex: 1,
    backgroundColor: '#FFF5F7',
  },
  scrollView: {
    flex: 1,
  },
  headerText: {
    fontSize: 24,
    fontWeight: 'bold',
    color: '#4A5568',
    marginVertical: 16,
    textAlign: 'center',
  },
  section: {
    padding: 16,
    marginBottom: 16,
  },
  sectionHeader: {
    flexDirection: 'row',
    justifyContent: 'space-between',
    alignItems: 'center',
    marginBottom: 16,
  },
  sectionTitle: {
    fontSize: 18,
    fontWeight: '600',
    color: '#4A5568',
  },
  flashbackCard: {
    backgroundColor: '#FFFFFF',
    borderRadius: 16,
    padding: 16,
    marginBottom: 12,
    shadowColor: '#718096',
    shadowOffset: { width: 0, height: 2 },
    shadowOpacity: 0.1,
    shadowRadius: 4,
  },
  flashbackDate: {
    fontSize: 14,
    color: '#9F7AEA',
    fontWeight: '600',
    marginBottom: 8,
  },
  flashbackContent: {
    fontSize: 16,
    color: '#4A5568',
    marginBottom: 8,
  },
  tipCard: {
    backgroundColor: '#FFFFFF',
    borderRadius: 16,
    padding: 16,
    marginBottom: 12,
    shadowColor: '#718096',
    shadowOffset: { width: 0, height: 2 },
    shadowOpacity: 0.1,
    shadowRadius: 4,
  },
  tipEmoji: {
    fontSize: 24,
    marginBottom: 8,
    textAlign: 'center',
  },
  tipTitle: {
    fontSize: 16,
    fontWeight: '600',
    color: '#4A5568',
    marginBottom: 8,
    textAlign: 'center',
  },
  tipMessage: {
    fontSize: 14,
    color: '#718096',
    lineHeight: 20,
  },
  categoryTag: {
    backgroundColor: '#F7FAFC',
    paddingHorizontal: 12,
    paddingVertical: 6,
    borderRadius: 12,
    alignSelf: 'flex-start',
  },
  categoryText: {
    fontSize: 14,
    color: '#718096',
  },
  emptyText: {
    textAlign: 'center',
    color: '#718096',
    fontSize: 14,
    fontStyle: 'italic',
  },
});
```

### G. app/(tabs)/wisdombook.tsx